# From Statistical Physics to Data-Driven Modelling with Applications to Quantitative Biology

Tutorial 4. Sparse estimation techniques for alternative splicing in gene expression.

Artificial data generated by the authors.

Starting Notebook SC RM FZ.



In [6]:
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix
import numpy.matlib
from random import randint
#Pip3 install sklearn 
from sklearn import linear_model
%matplotlib inline
#plt.rcParams["font.family"] = "serif"
#plt.rcParams["mathtext.fontset"] = "dejavuserif"
#plt.rcParams["figure.figsize"] = (10, 8)
#plt.rcParams["font.size"] = 24

Read data for the overlapping and nonoverlapping reads.

In [7]:
useoverlap=False

#L= fixed number of exons in a gene
L=10
#ttl= Total number of possible transcripts 
ttL=2**L


if (useoverlap):
    y=np.loadtxt('./Data/dataNe2Nt5.dat',dtype=int)
else:
    y=np.loadtxt('./Data/dataNe2Nt5-nooverlap.dat',dtype=int)
print(y)
#Original set of transcripts and their corresponding expressions
thetaread=np.loadtxt('./Data/transcriptscpnbNe2Nt5.dat',dtype=int)
Nt=np.size(thetaread,0)
theta=np.zeros(ttL-1,dtype=int)

[671 344  26  40 671   0   0 275  26   0]


Write down the transcript matrix A. 

In [8]:
#Function to define the transcript matrix A both in the non-overlapping case and in the overlapping case

def Amatrix (overlap):
    if (overlap):
        Lp=int(L*(L+1)/2.)
    else:
        Lp=L
    Ar=np.zeros((Lp,2**L-1)).astype(int)
    for i in range(2**L-1):
        for j in range(L):
            Ar[j,i]=(i+1)>>j&1
        if (overlap):
            rpart=Ar[:,i]
            #locate the ones in the vector
            ind=np.where(rpart==1)[0].astype(int)
            nb1=np.size(ind)
            #put the ones for the junctions
            for ii in range (nb1-1):
                xi=ind[ii]
                xj=ind[ii+1]
                ip=( -np.power(xi,2)/2+xj+xi*(-3/2+L) ).astype(int)
                Ar[L+ip-1,i]=1
    return Ar

#Build the matrix A
A = Amatrix(useoverlap)


In [9]:
#Check on real numbers of transcript that the linear relationship is verified
for i in range(Nt):
    theta[thetaread[i,0]]=thetaread[i,1]
print(coo_matrix(theta))
ytest = np.dot(A,theta)
print(ytest)

  (0, 7)	14
  (0, 16)	327
  (0, 18)	344
  (0, 127)	275
  (0, 267)	26
[671 344  26  40 671   0   0 275  26   0]



Question 1: Program the Lasso routine to find the minimum of 1/(2L) (y - P θ)^2 + γ |θ|. 



Compare with the Lasso routine in Python.

In [11]:
#Lasso routine in Python.
def mylasso_python(P,y,gamma):
    clf = linear_model.Lasso(alpha=gamma,positive=True,fit_intercept=False,max_iter=100000)
    clf.fit(P, y) 
    return clf.coef_

Question 2: Discuss the solutions for θ given by the lasso algorithm for some values of γ = 0.01, 0.1, 1, and compare to the real vector θ.

Question 4: Study the performances of the Lasso algorithm as a function of γ .
Plot the Mean Square Error as a function of γ, plot the quadratic cost function of the solution θ as a function  as a function of γ, and the total number of of non-zero elements of θ as a function of γ for the 5 transcripts corresponding to the true transcripts (t = 7, 16, 18, 127, 267).